<a href="https://colab.research.google.com/github/safa-abidi/NLP-sentimentAnalysis-AraBert/blob/main/fake_news__self_training_semi_supervised_with_Naive_Bayes_%26_AraBert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# imports

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import xml.etree.ElementTree as ET
import os
import torch
from typing import Any, Dict, List, Callable, Optional, Tuple, Union

from sklearn.model_selection import train_test_split 
from sklearn.svm import SVC
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.semi_supervised import SelfTrainingClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score

import transformers
from transformers import BertModel, BertTokenizer

In [ ]:
from google.colab import drive
drive.mount('/content/drive') 

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Dataset: labelled and unlabled

In [ ]:
DATASETS_PATH = '/content/drive/MyDrive/'

%cd /content/drive/MyDrive/datasets

path = 'dataset_arabic_translated'
file_list = os.listdir(path)

data_frames =[]

for filename in file_list:
    if filename.endswith('.xlsx'):
        file_path = os.path.join(path, filename)
        data = pd.read_excel(file_path)
        data = data[data['Review'].notnull()]
        data_frames.append(data)
 
labeled_dataset = pd.concat(data_frames)

# Parse the XML file
tree = ET.parse('SemEval2016_arabic/AR_Hotels_Train_SB1.xml')
root = tree.getroot()

# Extract reviews from XML and add to list of tuples
reviews = []
for review in root.findall('Review'):
    review_text = ''
    for sentence in review.findall('sentences/sentence'):
        review_text += sentence.find('text').text.strip() + ' '
    reviews.append((review_text, -1))

# Create a pandas DataFrame from the list of tuples
unlabeled_dataset = pd.DataFrame(reviews, columns=['Review', 'Class'])

df = pd.concat([labeled_dataset, unlabeled_dataset], axis=0)
# Reset the index of the concatenated DataFrame
df = df.reset_index(drop=True)

#df.to_csv ("Merged_Arabic_Dataset.csv", index = None,header=True)


/content/drive/MyDrive/datasets


In [ ]:
print(labeled_dataset)

    Hotel name                                             Review Source  \
0      Affinia  فندق Affinia Chicago هو أحد أسوأ الفنادق التي ...  Mturk   
1      Affinia  مكثت في Affina Chicago للاحتفال بالذكرى السنوي...  Mturk   
2      Affinia  من الواضح أن فندق Affinia في شيكاغو يخدم ضيوف ...  Mturk   
3      Affinia  محبط للغاية في إقامتنا. هذا الفندق لا يشبه ما ...  Mturk   
4      Affinia  كان فندق Affinia في شيكاغو أحد أكثر الفنادق إث...  Mturk   
..         ...                                                ...    ...   
75  Ambassador  نحن نعيش أسفل المبنى من الفندق. لقد رتبنا إنشا...    Web   
76  Ambassador  كن مستعدا الفندق قديم. إنه في القائمة التاريخي...    Web   
77  Ambassador  أنا أحب Ambassador إيست ، صحيح أنها كبيرة في ا...    Web   
78  Ambassador  رائع!! من أين أبدأ؟ حجزت هذا الفندق من خلال مو...    Web   
79  Ambassador  لقد عدنا للتو من عطلة نهاية أسبوع شديدة البرود...    Web   

   Opinion  Class  Unnamed: 5 Hyatt  
0      نفي   خادع         NaN   NaN  
1      نفي 

In [ ]:
print(unlabeled_dataset)

                                                 Review  Class
0     أنصح بالنوم وليس تناول الطعام  موقع مثالي للإق...     -1
1     لمسة بحرية  جميل وظريفة، فندق تقليدي في المدين...     -1
2     من افضل فنادق المدينه المنوره انصح وبشده من ير...     -1
3     من افضل الفنادق التي تجعلك تشتاق الي العرده ال...     -1
4     فندق رائع الفندق اكثر من رائع وخصوصا مستوى الخ...     -1
...                                                 ...    ...
1834  اكثر من رائع كما في العنوان الفندق اكثر من رائ...     -1
1835  النظافة سيئة. حمام قذر. الموقع 10 دقائق إلى أق...     -1
1836  احصل على معاملة ملكية  أفضل ضيافة يمكن أن تحصل...     -1
1837  الفندق الأروع والأجمل في ماليزيا فندق ومنتجع ر...     -1
1838  أفضل إقامة في فندق صغير إقامتنا دامت لمدة يومي...     -1

[1839 rows x 2 columns]


In [ ]:
print(df)

     Hotel name                                             Review Source  \
0       Affinia  فندق Affinia Chicago هو أحد أسوأ الفنادق التي ...  Mturk   
1       Affinia  مكثت في Affina Chicago للاحتفال بالذكرى السنوي...  Mturk   
2       Affinia  من الواضح أن فندق Affinia في شيكاغو يخدم ضيوف ...  Mturk   
3       Affinia  محبط للغاية في إقامتنا. هذا الفندق لا يشبه ما ...  Mturk   
4       Affinia  كان فندق Affinia في شيكاغو أحد أكثر الفنادق إث...  Mturk   
...         ...                                                ...    ...   
3434        NaN  اكثر من رائع كما في العنوان الفندق اكثر من رائ...    NaN   
3435        NaN  النظافة سيئة. حمام قذر. الموقع 10 دقائق إلى أق...    NaN   
3436        NaN  احصل على معاملة ملكية  أفضل ضيافة يمكن أن تحصل...    NaN   
3437        NaN  الفندق الأروع والأجمل في ماليزيا فندق ومنتجع ر...    NaN   
3438        NaN  أفضل إقامة في فندق صغير إقامتنا دامت لمدة يومي...    NaN   

     Opinion Class  Unnamed: 5 Hyatt  
0        نفي  خادع         NaN   NaN

In [ ]:
## Encode the dataset labels
df['Label'] = df.apply(lambda x: 0 if x['Class'] == 'خادع' else 1 if x['Class'] == 'صادقة' else -1, axis=1)

In [ ]:
print(df)

     Hotel name                                             Review Source  \
0       Affinia  فندق Affinia Chicago هو أحد أسوأ الفنادق التي ...  Mturk   
1       Affinia  مكثت في Affina Chicago للاحتفال بالذكرى السنوي...  Mturk   
2       Affinia  من الواضح أن فندق Affinia في شيكاغو يخدم ضيوف ...  Mturk   
3       Affinia  محبط للغاية في إقامتنا. هذا الفندق لا يشبه ما ...  Mturk   
4       Affinia  كان فندق Affinia في شيكاغو أحد أكثر الفنادق إث...  Mturk   
...         ...                                                ...    ...   
3434        NaN  اكثر من رائع كما في العنوان الفندق اكثر من رائ...    NaN   
3435        NaN  النظافة سيئة. حمام قذر. الموقع 10 دقائق إلى أق...    NaN   
3436        NaN  احصل على معاملة ملكية  أفضل ضيافة يمكن أن تحصل...    NaN   
3437        NaN  الفندق الأروع والأجمل في ماليزيا فندق ومنتجع ر...    NaN   
3438        NaN  أفضل إقامة في فندق صغير إقامتنا دامت لمدة يومي...    NaN   

     Opinion Class  Unnamed: 5 Hyatt  Label  
0        نفي  خادع         Na

In [ ]:
# data split
df_train, df_test = train_test_split(df, train_size=0.8, random_state=0)
print('Size of train dataframe: ', df_train.shape[0])
print('Size of test dataframe: ', df_test.shape[0])

Size of train dataframe:  2751
Size of test dataframe:  688


# BERT transformer

In [ ]:
class BertTransformer(BaseEstimator, TransformerMixin):
    def __init__(
        self,
        tokenizer,
        model,
        max_length: int = 60,
        embedding_func: Optional[Callable[[torch.Tensor], torch.Tensor]] = None,
    ):
        self.tokenizer = tokenizer
        self.model = model
        self.model.eval()
        self.max_length = max_length
        self.embedding_func = embedding_func

        if self.embedding_func is None:
            self.embedding_func = lambda x: x[0][:, 0, :].squeeze()

    def _tokenize(self, text: str) -> Tuple[torch.Tensor, torch.Tensor]:
        # Tokenize the text with the provided tokenizer
        tokenized_text = self.tokenizer.encode_plus(
            text, add_special_tokens=True, max_length=self.max_length,truncation=True
        )["input_ids"]
        
        # padding
        padded_text = tokenized_text + [0]*(self.max_length-len(tokenized_text))

        # Create an attention mask telling BERT to use all words and ignore padded values
        attention_mask = np.where(np.array(padded_text) != 0, 1, 0)

        # bert takes in a batch so we need to unsqueeze the rows
        return (
            torch.tensor(padded_text).unsqueeze(0),
            torch.tensor(attention_mask).unsqueeze(0),
        )

    def _tokenize_and_predict(self, text: str) -> torch.Tensor:
        tokenized, attention_mask = self._tokenize(text)

        embeddings = self.model(tokenized, attention_mask)
        return self.embedding_func(embeddings)

    def transform(self, text: List[str]):
        if isinstance(text, pd.Series):
            text = text.tolist()

        with torch.no_grad():
            return torch.stack([self._tokenize_and_predict(string) for string in text])

    def fit(self, X, y=None):
        """No fitting necessary so we just return ourselves"""
        return self
    
def calculate_classification_metrics(
    y_true: np.array,
    y_pred: np.array,
    average: Optional[str] = None,
    return_df: bool = True,
) -> Union[Dict[str, float], pd.DataFrame]:
    """Computes f1, precision, recall, precision, kappa, accuracy, and support

    Args:
        y_true: The true labels
        y_pred: The predicted labels
        average: How to average multiclass results

    Returns:
        Either a dataframe of the performance metrics or a single dictionary
    """
    labels = unique_labels(y_true, y_pred)

    # get results
    precision, recall, f_score, support = sk_metrics.precision_recall_fscore_support(
        y_true, y_pred, labels=labels, average=average
    )

    kappa = sk_metrics.cohen_kappa_score(y_true, y_pred, labels=labels)
    accuracy = sk_metrics.accuracy_score(y_true, y_pred)

    # create a pandas DataFrame
    if return_df:
        results = pd.DataFrame(
            {
                "class": labels,
                "f_score": f_score,
                "precision": precision,
                "recall": recall,
                "support": support,
                "kappa": kappa,
                "accuracy": accuracy,
            }
        )
    else:
        results = {
            "f1": f_score,
            "precision": precision,
            "recall": recall,
            "kappa": kappa,
            "accuracy": accuracy,
        }

    return results

## AraBERT embedding

In [ ]:
tokenizer = BertTokenizer.from_pretrained("aubmindlab/bert-base-arabertv2")
tokenized_dict = tokenizer.encode_plus(
    df_train["Review"].tolist(),
    add_special_tokens=True,
    max_length=512,  # Set maximum sequence length here
    truncation=True,
    )

bert_model = BertModel.from_pretrained("aubmindlab/bert-base-arabertv2")
tokenized_text = torch.tensor(tokenized_dict["input_ids"])
with torch.no_grad():
  embeddings = bert_model(torch.tensor(tokenized_text.unsqueeze(0)))

Some weights of the model checkpoint at aubmindlab/bert-base-arabertv2 were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
<ipython-input-29-37db532cadec>:12: UserWarning: To copy construct from a tensor, it is recommended to use sou

# Model classifier

In [ ]:
#from sklearn import naive bayes
from sklearn.naive_bayes import GaussianNB
from sklearn.pipeline import FeatureUnion, Pipeline

In [ ]:
X_train=df_train['Review']
y_train=df_train['Label'].values

X_test=df_test['Review']
y_test=df_test['Label'].values

bert_transformer = BertTransformer(tokenizer, bert_model)

## SVM model
#model_svc = SVC(kernel='rbf', 
                #probability=True,
                #C=1.0, 
                #gamma='scale', 
                #random_state=0
               #)
# Naive Bayes model
model_gnb = GaussianNB()

# Self training classifier
classifier = SelfTrainingClassifier(base_estimator=model_gnb,
                                             threshold=0.9,
                                             criterion='threshold',
                                             #k_best=50, 
                                             max_iter=100, 
                                             verbose=True
                                            )
model = Pipeline(
    [
        ("vectorizer", bert_transformer),
        ("classifier", classifier),
    ]
)

clf_ST = model.fit(X_train, y_train)

End of iteration 1, added 1412 new labels.
End of iteration 2, added 62 new labels.


In [ ]:
print('---------- Self Training Model - Summary ----------')

print('Classes: ', clf_ST.classes_)

print('---------- Self Training Model - Evaluation on Test Data ----------')
#accuracy_score_ST = clf_ST.score(X_test, y_test)
#print('Accuracy Score: ', accuracy_score_ST)
y_pred = clf_ST.predict(X_test)
f1 = f1_score(y_test, y_pred, average='weighted')
print("F1 Score:", f1)

# Look at classification report to evaluate the model
print(classification_report(y_test, clf_ST.predict(X_test)))

---------- Self Training Model - Summary ----------
Classes:  [0 1]
---------- Self Training Model - Evaluation on Test Data ----------
F1 Score: 0.18090682248040083
              precision    recall  f1-score   support

          -1       0.00      0.00      0.00       363
           0       0.41      0.81      0.55       156
           1       0.17      0.38      0.23       169

    accuracy                           0.28       688
   macro avg       0.19      0.40      0.26       688
weighted avg       0.13      0.28      0.18       688



/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
#graphe